In [19]:
# Keep columns: hashed_id, hashed_email, zip_code

import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
import json
from haversine import haversine

os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4'

In [21]:
samplerows=None
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [3]:
dfiddetail=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows=samplerows)
dfiddetail=dfiddetail.drop_duplicates('customer_id_hashed')
###
master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_2018Q1_2019Q1.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_2018Q1_2019Q1
###
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_gap.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_gap
###



In [4]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

all_master_files_after_201905=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201905=[x for x in all_master_files_after_201905 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201905=[x for x in all_master_files_after_201905 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-05-01"]
all_master_files_after_201905.reverse()
df_master_all_later_after_201905=pd.DataFrame()
for file in all_master_files_after_201905:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
    df_master_all_later_after_201905=df_master_all_later_after_201905.append(df)

dfiddetail=df_master_all_later_after_201905.append(dfiddetail).drop_duplicates("customer_id_hashed")
del df_master_all_later_after_201905

#dropped id duplicated
###
gc.collect()
print("dfiddetail.shape",dfiddetail.shape)

dfiddetail.shape (29802865, 3)


In [5]:
dfiddetail2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])

dfiddetail = dfiddetail.append(dfiddetail2,ignore_index = True)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
del dfiddetail2
gc.collect()

14

In [8]:
dfiddetail.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code
0,5ef4d1f4d0bc545e81340e8ad58db181e904392172c05d...,4979adf6b28136f0f0e7fd092b7f20712fe93fe8b21516...,NaN
1,9c95a62baf8dcc9c33b117853d30cdd516e219c34eeb6f...,8da7290d6a194661c5af68946ff9ff95ec310dabcb9569...,NaN


In [11]:
all_master_files_after_201905[0]

'/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-12/MediaStormMasterWeekly20191015-111356-088.txt'

In [12]:
# dfiddetail.to_csv("./BL_all_master_id_email_zip_upto_20191012_JL_"+str(datetime.datetime.now().date()),index=False)

In [13]:
#dropped both email & id duplicated


In [23]:
dfiddetail['customer_zip_code']=dfiddetail['customer_zip_code'].fillna("nan")
dfiddetail['cleaned_zip_cd']=dfiddetail['customer_zip_code'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5)[:5])

In [63]:
print(dfiddetail.shape)

(30732606, 4)


# Closest_Store

In [46]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",dtype=str,sep="|")
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]
store_list=store_list[['location_id','latitude_meas','longitude_meas']]

store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)


In [47]:
dict_store_list=store_list.copy()
dict_store_list['lat_lng']=dict_store_list[['latitude_meas','longitude_meas']].values.tolist()
dict_store_list=dict_store_list.set_index("location_id").to_dict()["lat_lng"]

In [25]:
dfiddetail_unique_zips=dfiddetail[['cleaned_zip_cd']].drop_duplicates()
dfiddetail_unique_zips=dfiddetail_unique_zips[pd.notnull(dfiddetail_unique_zips['cleaned_zip_cd'])]
dfiddetail_unique_zips=dfiddetail_unique_zips[dfiddetail_unique_zips['cleaned_zip_cd']!="00nan"]

print(dfiddetail_unique_zips.shape)

(69013, 1)


In [26]:
dfiddetail_unique_zips['cleaned_zip_cd'].apply(len).unique()

array([5])

In [60]:
def find_the_closest_store(x_zip_cd):
    dist_min=999999999
    if x_zip_cd not in zip_centers.keys():
        return np.nan,np.nan
    else:
        zip_center=zip_centers[x_zip_cd]
        for location_id,store_center in dict_store_list.items():
            dist=haversine(zip_center,store_center,unit="mi")
            if dist<dist_min:
                dist_min=dist
                closet_store=location_id
    return closet_store,dist_min
                

In [61]:
dfiddetail_unique_zips['closest_store']=dfiddetail_unique_zips['cleaned_zip_cd'].apply(lambda x: find_the_closest_store(x)[0])
dfiddetail_unique_zips['closest_distance']=dfiddetail_unique_zips['cleaned_zip_cd'].apply(lambda x: find_the_closest_store(x)[1])



In [64]:
dfiddetail=pd.merge(dfiddetail,dfiddetail_unique_zips,on="cleaned_zip_cd",how="left")
dfiddetail.shape

(30732606, 6)

In [65]:
dfiddetail.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,cleaned_zip_cd,closest_store,closest_distance
0,5ef4d1f4d0bc545e81340e8ad58db181e904392172c05d...,4979adf6b28136f0f0e7fd092b7f20712fe93fe8b21516...,nan,00nan,NaN,NaN
1,9c95a62baf8dcc9c33b117853d30cdd516e219c34eeb6f...,8da7290d6a194661c5af68946ff9ff95ec310dabcb9569...,nan,00nan,NaN,NaN


In [66]:
dfiddetail[pd.notnull(dfiddetail['closest_store'])].shape

(26936791, 6)

In [67]:
26936791/30732606

0.8764889967352589

# Zip_Label

In [2]:
import pandas as pd
Excel_TA=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
df_zip_in_TA=Excel_TA.parse("unique_zips_full_footprint",dtype=str)
df_store_in_TA=Excel_TA.parse("view_by_store",dtype=str)
stores_new=store_list[~store_list['location_id'].isin(df_store_in_TA['location_id'].tolist())].reset_index()
del stores_new['index']

In [87]:
print("df_zip_in_TA['zip_cd'].nunique()",df_zip_in_TA['zip_cd'].nunique())
print("df_zip_in_TA.shape",df_zip_in_TA['zip_cd'].shape)

print("df_zip_in_TA['zip_type'].nunique()",df_zip_in_TA['zip_type'].unique())

df_zip_in_TA.head(2)


df_zip_in_TA['zip_cd'].nunique() 12980
df_zip_in_TA.shape (12980,)
df_zip_in_TA['zip_type'].nunique() ['trans_P' 'trans_S' 'zips_10']


,zip_type,zip_cd
0,trans_P,75040
1,trans_P,20743


In [90]:
df_new_store_zips=pd.DataFrame()
for ind,row in stores_new.iterrows():
    store_num=row['location_id']
    store_center=(row['latitude_meas'],row['longitude_meas'])
    df_store_zip10=pd.DataFrame()
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi")
        if dist<=10:
            df=pd.DataFrame({"dist":dist},index=[zip_cd])
            df_store_zip10=df_store_zip10.append(df)
    df_store_zip10['location_id']= store_num     
    df_store_zip10=df_store_zip10.reset_index().rename(columns={"index":"zip_cd"})
    df_new_store_zips=df_new_store_zips.append(df_store_zip10)
df_new_store_zips['zip_type']="zips_10"
df_new_store_zips.shape

(404, 4)

In [92]:
df_zip_in_TA.head(2)

,zip_type,zip_cd
0,trans_P,75040
1,trans_P,20743


In [93]:
df_new_store_zips.head(2)

,zip_cd,dist,location_id,zip_type
0,82001,1.719839,4678,zips_10
1,82009,3.949026,4678,zips_10


In [94]:
df_to_append_new_store_zips=df_new_store_zips[['zip_cd','zip_type']].drop_duplicates()
df_zip_in_TA=df_zip_in_TA.append(df_to_append_new_store_zips)
df_zip_in_TA=df_zip_in_TA.sort_values(["zip_cd","zip_type"]).drop_duplicates("zip_cd")

In [102]:
df_zip_in_TA=df_zip_in_TA.rename(columns={"zip_cd":"cleaned_zip_cd"})

print(df_zip_in_TA.shape,df_zip_in_TA['cleaned_zip_cd'].nunique())

(13201, 2) 13201


In [103]:
print(dfiddetail.shape)
dfiddetail=pd.merge(dfiddetail,df_zip_in_TA,on="cleaned_zip_cd",how="left")
print(dfiddetail.shape)

(30732606, 6)
(30732606, 7)


In [104]:
dfiddetail.to_csv("./BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_"+str(datetime.datetime.now().date()),index=False)

In [105]:
dfiddetail.shape

(30732606, 7)